In [24]:
import pandas as pd

# Load your product-level report
product_df = pd.read_csv("order-with-product-report.csv", encoding='ISO-8859-1')

# Load your box-level report
box_df = pd.read_csv("order-with-box-report.csv", encoding='ISO-8859-1')

# Preview
print("Product Report:")
print(product_df.head())

print("\nBox Report:")
print(box_df.head())

In [25]:
# !pip3 install sqlalchemy psycopg2-binary



In [26]:
# !pip3 install psycopg2-binary


In [27]:
# import sys
# !{sys.executable} -m pip install psycopg2-binary

In [27]:
# import pandas as pd
# from sqlalchemy import create_engine
# from urllib.parse import quote_plus

# # Define DB credentials
# user = "digvijaykasana"
# raw_password = "Mydog@1234"
# password = quote_plus(raw_password)  
# host = "localhost"
# port = "5432"
# dbname = "ev_orders"

# # Create the engine with encoded password
# engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")




In [28]:
product_df = pd.read_sql("SELECT * FROM product_report", con=engine)
print("📦 Product Data:")
print(product_df.head())

# ✅ Load box_report table from PostgreSQL
box_df = pd.read_sql("SELECT * FROM box_report", con=engine)
print("📦 Box Data:")
print(box_df.head())

📦 Product Data:
            Order No.                Product Id  \
0  ORD-20230125-23461  60af8e8d6846678a4076cc11   
1  ORD-20230131-23750  62de9b9b684667e8ab14ea0d   
2                None  6156d7e0684667bb9c141cef   
3                None  62b160f9684667814b6c640f   
4                None  6360e0426846671c6649541a   

                                                     Product Name Category  \
0                       Hydravol Pre Workout Powder | Lemon 400 G   1100ml   
1                                                 Qnt Cla 30 Caps     None   
2                               Maxi L-carnitine 1000mg | 60 Tabs     None   
3  Prime Whey Triple Blend Isolate Protein | Irish Chocolate 1 Kg   4240cc   
4                        Prime Whey New Pack Choco-badam 1 Kg 1 L   4240cc   

   Quantity  Product Weight  
0         1           0.514  
1         1           0.100  
2         5           0.136  
3         4           1.192  
4         6             NaN  
📦 Box Data:
            Orde

In [29]:
# Step 1: Clean product_df
product_df['Order No.'] = product_df['Order No.'].ffill()
product_df['Category'] = product_df['Category'].fillna('100cc')

# Clean Quantity & Product Weight
product_df['Quantity'] = pd.to_numeric(product_df['Quantity'], errors='coerce').fillna(0).astype(int)
product_df['Product Weight'] = pd.to_numeric(product_df['Product Weight'], errors='coerce')

# Drop empty product name rows
product_df.dropna(subset=['Product Name'], inplace=True)

# Preview cleaned product_df
print("✅ Cleaned Product DataFrame:")
print(product_df.head(20))


✅ Cleaned Product DataFrame:
             Order No.                Product Id  \
0   ORD-20230125-23461  60af8e8d6846678a4076cc11   
1   ORD-20230131-23750  62de9b9b684667e8ab14ea0d   
2   ORD-20230131-23750  6156d7e0684667bb9c141cef   
3   ORD-20230131-23750  62b160f9684667814b6c640f   
4   ORD-20230131-23750  6360e0426846671c6649541a   
5   ORD-20230131-23750  63c3a7936846679a341a5247   
6   ORD-20230131-23750  60af8e8d6846678a4076cbe0   
7   ORD-20230131-23750  63455d66684667a3f878e115   
8   ORD-20230131-23750  60af8e8d6846678a4076cbe0   
9   ORD-20230131-23750  63c3a7936846679a341a5247   
10  ORD-20230131-23750  61c16c01684667987972b78c   
11  ORD-20230131-23750  61c16c01684667987972b78e   
12  ORD-20230131-23750  61c16c01684667987972b790   
13  ORD-20230131-23750  62b160f9684667814b6c6419   
14  ORD-20230131-23750  62b160f9684667814b6c6411   
15  ORD-20230131-23750  62b160f9684667814b6c640f   
16  ORD-20230131-23750  62b07e876846672aee3068ce   
17  ORD-20230307-25944  611e200e684

In [30]:
# Step 2: Clean and Pivot box_df
import numpy as np

# 1. Forward-fill 'Order No.' and total-level fields
box_df['Order No.'] = box_df['Order No.'].ffill()
box_df[['Total No.Of Boxes', 'Total No.Of Products', 'Total No. Of Quantity']] = box_df[[
    'Total No.Of Boxes', 'Total No.Of Products', 'Total No. Of Quantity'
]].ffill()

# 2. Convert relevant columns to numeric
numeric_cols = ['Total No.Of Boxes', 'Total No.Of Products', 'Total No. Of Quantity',
                'Box Length', 'Box Breadth', 'Box Height', 'Box Weight', 'Box']
box_df[numeric_cols] = box_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# 3. Drop rows missing any of the box dimension
box_df.dropna(subset=['Box Length', 'Box Breadth', 'Box Height', 'Box'], inplace=True)

# 4. Create 'Box Dimension' column like "40x40x34"
box_df['Box Dimension'] = (
    box_df['Box Length'].astype(int).astype(str) + 'x' +
    box_df['Box Breadth'].astype(int).astype(str) + 'x' +
    box_df['Box Height'].astype(int).astype(str)
)

# 5. Pivot: Create columns like '40x40x34' with values = number of boxes
box_pivot = box_df.pivot_table(index='Order No.', 
                               columns='Box Dimension', 
                               values='Box', 
                               aggfunc='sum', 
                               fill_value=0)

# 6. Get unique box info per order (1 row per order with totals)
box_meta = box_df[['Order No.', 'Total No.Of Boxes', 'Total No.Of Products', 'Total No. Of Quantity']].drop_duplicates()

# 7. Merge total info with pivoted box counts
final_box_df = pd.merge(box_meta, box_pivot, on='Order No.', how='left')

# Preview
print("✅ Final Box-Level Structured Data:")
print(final_box_df.head(20))


✅ Final Box-Level Structured Data:
              Order No.  Total No.Of Boxes  Total No.Of Products  \
0    ORD-20230125-23461                1.0                   1.0   
1    ORD-20230131-23750                1.0                  16.0   
2    ORD-20230307-25944                1.0                   1.0   
3    ORD-20230317-26450                1.0                   2.0   
4    ORD-20230923-37441                1.0                   1.0   
5    ORD-20231129-41619                1.0                   1.0   
6    ORDB-20240627-2533               47.0                  26.0   
7    ORDB-20250210-7623                5.0                   8.0   
8   ORDG-20250218-18960                1.0                   5.0   
9    ORDG-20250430-1719                1.0                   3.0   
10    ORDB-20250527-729                1.0                   4.0   
11   ORDG-20250527-3525                1.0                   1.0   
12   ORDG-20250528-3559                1.0                   1.0   
13    ORDB-20

In [31]:
# Step 3: Clean and Aggregate product_df

# 1. Forward-fill 'Order No.'
product_df['Order No.'] = product_df['Order No.'].ffill()

# 2. Fill missing 'Category' with "100 cc"
product_df['Category'] = product_df['Category'].fillna('100 cc')

# 3. Fill missing 'Quantity' with 0 (and convert to int)
product_df['Quantity'] = pd.to_numeric(product_df['Quantity'], errors='coerce').fillna(0).astype(int)

# 4. Normalize text fields
product_df['Category'] = product_df['Category'].astype(str).str.strip()

# 5. Aggregate per order (drop Product Name aggregation, keep Category and Quantity)
product_agg_df = product_df.groupby('Order No.').agg({
    'Product Id': 'nunique',
    'Quantity': ['sum', 'mean', 'max', list],
    'Category': lambda x: list(x)
}).reset_index()

# 6. Flatten column names
product_agg_df.columns = [
    'Order No.', 
    'Unique Product Count', 
    'Total Quantity', 
    'Avg Quantity', 
    'Max Quantity', 
    'Quantities', 
    'Categories'
]

# Preview
print("✅ Aggregated Product-Level Data (without Product Names):")
print(product_agg_df.head(20))


✅ Aggregated Product-Level Data (without Product Names):
             Order No.  Unique Product Count  Total Quantity  Avg Quantity  \
0   ORD-20230125-23461                     1               1      1.000000   
1   ORD-20230131-23750                    13              97      6.062500   
2   ORD-20230307-25944                     1               1      1.000000   
3   ORD-20230317-26450                     2               2      1.000000   
4   ORD-20230923-37441                     1               1      1.000000   
5   ORD-20231129-41619                     1               1      1.000000   
6   ORDB-20240627-2533                    26             319     12.269231   
7   ORDB-20250210-7623                     8              64      8.000000   
8    ORDB-20250527-729                     4               6      1.500000   
9    ORDB-20250531-811                    13             242     18.615385   
10   ORDB-20250605-907                     7              20      2.857143   
11   OR

In [32]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Don't truncate horizontally
pd.set_option('display.max_colwidth', None)  # Show full column content

# Now print or display
display(product_agg_df)  # preferred in Jupyter Notebook

,Order No.,Unique Product Count,Total Quantity,Avg Quantity,Max Quantity,Quantities,Categories
0,ORD-20230125-23461,1,1,1.000000,1,[1],[1100ml]
1,ORD-20230131-23750,13,97,6.062500,12,"[1, 5, 4, 6, 3, 2, 4, 12, 12, 12, 9, 9, 4, 4, 6, 4]","[100cc, 100cc, 4240cc, 4240cc, 100cc, 100cc, 100cc, 100cc, 100cc, 500G, 500G, 500G, 4240cc, 4240cc, 4240cc, 8150ml]"
2,ORD-20230307-25944,1,1,1.000000,1,[1],[4240cc]
3,ORD-20230317-26450,2,2,1.000000,1,"[1, 1]","[5lbs, 100cc]"
4,ORD-20230923-37441,1,1,1.000000,1,[1],[8150ml]
5,ORD-20231129-41619,1,1,1.000000,1,[1],[8150ml]
6,ORDB-20240627-2533,26,319,12.269231,48,"[1, 48, 24, 24, 2, 1, 5, 5, 5, 10, 10, 10, 4, 2, 12, 12, 24, 24, 12, 12, 12, 12, 6, 6, 12, 24]","[100cc, 100cc, 710ml, 710ml, 8150ml, 4240cc, 100cc, 100cc, 100cc, 100cc, 100cc, 100cc, 12 weight|kg, 12 weight|kg, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml, 4240cc, 4240cc, 4240cc, 4240cc]"
7,ORDB-20250210-7623,8,64,8.000000,24,"[24, 24, 2, 2, 3, 2, 3, 4]","[100cc, 100cc, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml]"
8,ORDB-20250527-729,4,6,1.500000,2,"[2, 2, 1, 1]","[500G, 100cc, 100cc, 4240cc]"
9,ORDB-20250531-811,13,242,18.615385,100,"[36, 26, 20, 100, 4, 4, 4, 4, 4, 4, 12, 12, 12]","[150g, 100cc, 500G, 900ml, 8150ml, 8150ml, 8150ml, 12 weight|kg, 12 weight|kg, 12 weight|kg, 8150ml, 8150ml, 8150ml]"


In [33]:
# Step 4 – Merge box pivot with product aggregation on 'Order No.'
final_dataset = pd.merge(
    final_box_df,        # Contains per-order box dimensions (target columns)
    product_agg_df,      # Contains per-order product info (features)
    on='Order No.',
    how='left'
)


In [34]:
# Step 4 – Fix missing values correctly (cleaned and relevant)

# 4.1: Fill numeric column
final_dataset['Unique Product Count'] = final_dataset['Unique Product Count'].fillna(0)

# 4.2: Fill list-based columns (excluding dropped Product Names)
final_dataset['Categories'] = final_dataset['Categories'].apply(lambda x: x if isinstance(x, list) else [])
final_dataset['Quantities'] = final_dataset['Quantities'].apply(lambda x: x if isinstance(x, list) else [])

# 4.3: Fill box count columns with 0
box_cols = [col for col in final_dataset.columns if 'x' in col and 'cm' in col]
final_dataset[box_cols] = final_dataset[box_cols].fillna(0)

# Final check
print("✅ Cleaned dataset shape:", final_dataset.shape)
final_dataset.head(20)


✅ Cleaned dataset shape: (277, 26)


,Order No.,Total No.Of Boxes,Total No.Of Products,Total No. Of Quantity,10x10x10,12x18x10,12x8x20,18x17x27,20x20x34,30x30x29,32x20x17,38x30x20,40x30x29,40x30x34,40x40x34,42x42x30,48x34x27,60x30x30,8x22x13,8x8x13,Unique Product Count,Total Quantity,Avg Quantity,Max Quantity,Quantities,Categories
0,ORD-20230125-23461,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1.000000,1,[1],[1100ml]
1,ORD-20230131-23750,1.0,16.0,97.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,97,6.062500,12,"[1, 5, 4, 6, 3, 2, 4, 12, 12, 12, 9, 9, 4, 4, 6, 4]","[100cc, 100cc, 4240cc, 4240cc, 100cc, 100cc, 100cc, 100cc, 100cc, 500G, 500G, 500G, 4240cc, 4240cc, 4240cc, 8150ml]"
2,ORD-20230307-25944,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1.000000,1,[1],[4240cc]
3,ORD-20230317-26450,1.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,1.000000,1,"[1, 1]","[5lbs, 100cc]"
4,ORD-20230923-37441,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1.000000,1,[1],[8150ml]
5,ORD-20231129-41619,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1.000000,1,[1],[8150ml]
6,ORDB-20240627-2533,47.0,26.0,319.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,18.0,2.0,12.0,6.0,0.0,0.0,26,319,12.269231,48,"[1, 48, 24, 24, 2, 1, 5, 5, 5, 10, 10, 10, 4, 2, 12, 12, 24, 24, 12, 12, 12, 12, 6, 6, 12, 24]","[100cc, 100cc, 710ml, 710ml, 8150ml, 4240cc, 100cc, 100cc, 100cc, 100cc, 100cc, 100cc, 12 weight|kg, 12 weight|kg, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml, 4240cc, 4240cc, 4240cc, 4240cc]"
7,ORDB-20250210-7623,5.0,8.0,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,8,64,8.000000,24,"[24, 24, 2, 2, 3, 2, 3, 4]","[100cc, 100cc, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml, 8150ml]"
8,ORDG-20250218-18960,1.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,6,1.200000,2,"[1, 1, 1, 1, 2]","[100cc, 900ml, 4240cc, 100cc, 4240cc]"
9,ORDG-20250430-1719,1.0,3.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3,1.000000,1,"[1, 1, 1]","[100cc, 4240cc, 100cc]"


In [35]:
import ast
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer

# ✅ Step 5.1: Safely convert stringified lists to actual Python lists
def safe_literal_eval(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except:
            return [val]  # fallback if malformed
    return val  # already a list

final_dataset['Quantities'] = final_dataset['Quantities'].apply(safe_literal_eval)
final_dataset['Categories'] = final_dataset['Categories'].apply(safe_literal_eval)

# ✅ Step 5.2: Build weighted quantity maps for Categories only
def build_weighted_dict(keys, weights):
    weight_map = {}
    if not isinstance(keys, list) or not isinstance(weights, list):
        return weight_map
    for k, w in zip(keys, weights):
        key = str(k).strip()
        if key in weight_map:
            weight_map[key] += w
        else:
            weight_map[key] = w
    return weight_map

final_dataset['Category_Weight_Map'] = final_dataset.apply(
    lambda row: build_weighted_dict(row['Categories'], row['Quantities']), axis=1
)

# ✅ Step 5.3: Vectorize category-weighted encoding
category_vectorizer = DictVectorizer(sparse=False)
category_features = pd.DataFrame(
    category_vectorizer.fit_transform(final_dataset['Category_Weight_Map']),
    columns=[f"Category_{feat}" for feat in category_vectorizer.get_feature_names_out()],
    index=final_dataset.index
)

# ✅ Step 5.4: Merge and clean up
final_dataset = pd.concat([final_dataset, category_features], axis=1)

final_dataset.drop(['Categories', 'Quantities', 'Category_Weight_Map'], axis=1, inplace=True)


# ✅ Drop redundant columns
final_dataset.drop(['Unique Product Count', 'Total Quantity', 'Total No.Of Boxes', 'Total No.Of Products'], axis=1, inplace=True)

# ✅ Verify
print("✅ Columns dropped. Updated shape:", final_dataset.shape)


# ✅ Preview
print("✅ Final dataset shape after weighted category encoding:", final_dataset.shape)
final_dataset.head(100)


✅ Columns dropped. Updated shape: (277, 31)
✅ Final dataset shape after weighted category encoding: (277, 31)


,Order No.,Total No. Of Quantity,10x10x10,12x18x10,12x8x20,18x17x27,20x20x34,30x30x29,32x20x17,38x30x20,40x30x29,40x30x34,40x40x34,42x42x30,48x34x27,60x30x30,8x22x13,8x8x13,Avg Quantity,Max Quantity,Category_100cc,Category_1100ml,Category_12 weight|kg,Category_150g,Category_4240cc,Category_500G,Category_5lbs,Category_710ml,Category_8150ml,Category_900ml,Category_C box 16*12*11.5
0,ORD-20230125-23461,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ORD-20230131-23750,97.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.062500,12,39.0,0.0,0.0,0.0,24.0,30.0,0.0,0.0,4.0,0.0,0.0
2,ORD-20230307-25944,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ORD-20230317-26450,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,ORD-20230923-37441,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,ORD-20231129-41619,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,ORDB-20240627-2533,319.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,18.0,2.0,12.0,6.0,0.0,0.0,12.269231,48,94.0,0.0,6.0,0.0,49.0,0.0,0.0,48.0,122.0,0.0,0.0
7,ORDB-20250210-7623,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,8.000000,24,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0
8,ORDG-20250218-18960,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.200000,2,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0
9,ORDG-20250430-1719,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# final_dataset.to_csv('final_dataset.csv', index=False)

In [37]:
from sklearn.model_selection import train_test_split
import numpy as np

# target box dimension columns
box_columns = [
    '10x10x10', '12x18x10', '12x8x20', '18x17x27', '20x20x34',
    '30x30x29', '32x20x17', '38x30x20', '40x30x29', '40x30x34',
    '40x40x34', '42x42x30', '48x34x27', '60x30x30', '8x22x13', '8x8x13'
]

# Define features and target
X = final_dataset.drop(columns=box_columns + ['Order No.'])
X = X.select_dtypes(include=[float, int])  # Keep only numeric columns
y = final_dataset[box_columns]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ Feature matrix shape:", X.shape)
print("✅ Target matrix shape:", y.shape)
print("✅ Training set shape:", X_train.shape, y_train.shape)
print("✅ Test set shape:", X_test.shape, y_test.shape)


✅ Feature matrix shape: (277, 14)
✅ Target matrix shape: (277, 16)
✅ Training set shape: (221, 14) (221, 16)
✅ Test set shape: (56, 14) (56, 16)


In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

# Random Forest model
rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
rf.fit(X_train, y_train)

# Predict
rf_preds = rf.predict(X_test)

# Evaluate
rf_rmse_scores = np.sqrt(mean_squared_error(y_test, rf_preds, multioutput='raw_values'))
rf_avg_rmse = np.mean(rf_rmse_scores)

# Display results
box_rmse_rf = pd.DataFrame({
    'Box Dimension': y.columns,
    'RMSE': rf_rmse_scores
})
print("📦 Random Forest - Per-Box RMSE Scores:\n")
print(box_rmse_rf)
print(f"\n📊 Random Forest - Average RMSE across all boxes: {round(rf_avg_rmse, 2)}")


📦 Random Forest - Per-Box RMSE Scores:

   Box Dimension      RMSE
0       10x10x10  0.147559
1       12x18x10  0.218314
2        12x8x20  0.017372
3       18x17x27  0.118453
4       20x20x34  0.263409
5       30x30x29  0.788255
6       32x20x17  0.208198
7       38x30x20  0.428640
8       40x30x29  0.415086
9       40x30x34  1.115650
10      40x40x34  1.037035
11      42x42x30  0.260580
12      48x34x27  0.677972
13      60x30x30  0.633737
14       8x22x13  0.179790
15        8x8x13  0.060618

📊 Random Forest - Average RMSE across all boxes: 0.41


In [39]:
# pip install optuna


In [40]:
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

def rf_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'max_depth': trial.suggest_int('max_depth', 5, 40),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    }

    rf = RandomForestRegressor(**params, random_state=42, n_jobs=-1)
    model = MultiOutputRegressor(rf)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds, multioutput='raw_values'))
    return np.mean(rmse)

rf_study = optuna.create_study(direction='minimize')
rf_study.optimize(rf_objective, n_trials=30)

best_rf_params = rf_study.best_params
print(" Best Random Forest Params:", best_rf_params)

final_rf = MultiOutputRegressor(RandomForestRegressor(**best_rf_params, random_state=42, n_jobs=-1))
final_rf.fit(X_train, y_train)
rf_preds = final_rf.predict(X_test)

rf_rmse_per_box = np.sqrt(mean_squared_error(y_test, rf_preds, multioutput='raw_values'))
rf_rmse_df = pd.DataFrame({
    "Box Dimension": box_columns,
    "RMSE": rf_rmse_per_box
})
rf_rmse_df.loc[len(rf_rmse_df.index)] = ["Average", np.mean(rf_rmse_per_box)]

print(" Optuna-Tuned Random Forest - Per-Box RMSE:")
print(rf_rmse_df)


[I 2025-08-29 17:25:13,927] A new study created in memory with name: no-name-58cf7c48-46b2-4f35-97ca-8211436b77d8
[I 2025-08-29 17:25:15,935] Trial 0 finished with value: 0.412087587028794 and parameters: {'n_estimators': 236, 'max_depth': 27, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': None}. Best is trial 0 with value: 0.412087587028794.
[I 2025-08-29 17:25:19,675] Trial 1 finished with value: 0.46058131430549154 and parameters: {'n_estimators': 464, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 'log2'}. Best is trial 0 with value: 0.412087587028794.
[I 2025-08-29 17:25:23,057] Trial 2 finished with value: 0.4114069991257784 and parameters: {'n_estimators': 421, 'max_depth': 12, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': None}. Best is trial 2 with value: 0.4114069991257784.
[I 2025-08-29 17:25:27,579] Trial 3 finished with value: 0.42090531219899363 and parameters: {'n_estimators': 573, 'max_depth': 29, 'min_sa

🌲 Best Random Forest Params: {'n_estimators': 796, 'max_depth': 13, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': None}
🌲 Optuna-Tuned Random Forest - Per-Box RMSE:
   Box Dimension      RMSE
0       10x10x10  0.134700
1       12x18x10  0.224425
2        12x8x20  0.033843
3       18x17x27  0.127499
4       20x20x34  0.236765
5       30x30x29  0.858044
6       32x20x17  0.125442
7       38x30x20  0.513675
8       40x30x29  0.357731
9       40x30x34  1.115373
10      40x40x34  1.166057
11      42x42x30  0.261639
12      48x34x27  0.355508
13      60x30x30  0.439793
14       8x22x13  0.172850
15        8x8x13  0.053231
16       Average  0.386036


In [ ]:
# from xgboost import XGBRegressor

# # XGBoost model
# xgb = MultiOutputRegressor(XGBRegressor(n_estimators=100, random_state=42, verbosity=0))
# xgb.fit(X_train, y_train)

# # Predict
# xgb_preds = xgb.predict(X_test)

# # Evaluate
# xgb_rmse_scores = np.sqrt(mean_squared_error(y_test, xgb_preds, multioutput='raw_values'))
# xgb_avg_rmse = np.mean(xgb_rmse_scores)

# # Display results
# box_rmse_xgb = pd.DataFrame({
#     'Box Dimension': y.columns,
#     'RMSE': xgb_rmse_scores
# })
# print("📦 XGBoost - Per-Box RMSE Scores:\n")
# print(box_rmse_xgb)
# print(f"\n📊 XGBoost - Average RMSE across all boxes: {round(xgb_avg_rmse, 2)}")


In [ ]:
# from xgboost import XGBRegressor

# def xgb_objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
#         'max_depth': trial.suggest_int('max_depth', 3, 12),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
#         'gamma': trial.suggest_float('gamma', 0, 5),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
#     }

#     xgb = XGBRegressor(objective='reg:squarederror', **params, random_state=42, verbosity=0)
#     model = MultiOutputRegressor(xgb)
#     model.fit(X_train, y_train)
#     preds = model.predict(X_test)
#     rmse = np.sqrt(mean_squared_error(y_test, preds, multioutput='raw_values'))
#     return np.mean(rmse)

# xgb_study = optuna.create_study(direction='minimize')
# xgb_study.optimize(xgb_objective, n_trials=30)

# best_xgb_params = xgb_study.best_params
# print("⚡ Best XGBoost Params:", best_xgb_params)

# final_xgb = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror', **best_xgb_params, random_state=42, verbosity=0))
# final_xgb.fit(X_train, y_train)
# xgb_preds = final_xgb.predict(X_test)

# xgb_rmse_per_box = np.sqrt(mean_squared_error(y_test, xgb_preds, multioutput='raw_values'))
# xgb_rmse_df = pd.DataFrame({
#     "Box Dimension": box_columns,
#     "RMSE": xgb_rmse_per_box
# })
# xgb_rmse_df.loc[len(xgb_rmse_df.index)] = ["Average", np.mean(xgb_rmse_per_box)]

# print("⚡ Optuna-Tuned XGBoost - Per-Box RMSE:")
# print(xgb_rmse_df)




Applied basic machine learning techniques using R Studio to analyze approximately 5,000 customer data points 
 
Contributed to predictive model development with the analytics team, helping improve forecast accuracy 
Assisted in creating a dashboard for monitoring 4 key business metrics using Power BI used by the management team 
 
Supported data migration tasks involving approximately 10,000 records, helping with data validation using SQL and Excel 
Collaborated with team members across different locations on analytics projects 
Helped prepare 3 reports summarizing findings for department managers 

In [ ]:
import joblib

# Save your Optuna-tuned Random Forest model
joblib.dump(final_rf, "optuna_random_forest_model2.pkl")
print("✅ Model saved as optuna_random_forest_model2.pkl")


In [25]:
# import sklearn
# print(sklearn.__version__)


In [52]:
import pandas as pd

# --- input in the same feature order your model expects ---
input_data = pd.DataFrame({
    "Total No. Of Quantity": [1217.0],
    "Avg Quantity": [110.64],
    "Max Quantity": [1000.0],

    "Category_100cc": [100.0],
    "Category_1100ml": [23.0],
    "Category_12 weight|kg": [1000.0],
    "Category_150g": [4.0],
    "Category_4240cc": [12.0],
    "Category_500G": [0.0],
    "Category_5lbs": [0.0],
    "Category_710ml": [54.0],
    "Category_8150ml": [12.0],
    "Category_900ml": [10.0],
    "Category_C box 16*12*11.5": [2.0]
})

# --- predict ---
y_pred = final_rf.predict(input_data)[0]   # take first row of predictions

# --- map prediction back to box dimensions ---
box_dimensions = [
    "10x10x10",
    "12x18x10",
    "12x8x20",
    "18x17x27",
    "20x20x34",
    "30x30x29",
    "32x20x17",
    "38x30x20",
    "40x30x29",
    "40x30x34",
    "40x40x34",
    "42x42x30",
    "48x34x27",
    "60x30x30",
    "8x22x13",
    "8x8x13"
]

predictions = {dim: int(round(val)) for dim, val in zip(box_dimensions, y_pred)}

print("📦 Predicted box counts per dimension:")
for dim, val in predictions.items():
    print(f"{dim}: {val}")


📦 Predicted box counts per dimension:
10x10x10: 0
12x18x10: 0
12x8x20: 0
18x17x27: 0
20x20x34: 0
30x30x29: 0
32x20x17: 0
38x30x20: 0
40x30x29: 0
40x30x34: 0
40x40x34: 0
42x42x30: 0
48x34x27: 0
60x30x30: 0
8x22x13: 0
8x8x13: 0


In [42]:
# pip install skl2onnx onnx


In [48]:
# import skl2onnx
# from skl2onnx import convert_sklearn
# from skl2onnx.common.data_types import FloatTensorType
# import onnx

# # Extract number of features directly from your X_train
# initial_type = [("float_input", FloatTensorType([None, X_train.shape[1]]))]

# # Convert your final_rf model to ONNX
# onnx_model = convert_sklearn(final_rf, initial_types=initial_type)

# # Save the ONNX model to file
# with open("optuna_rf_multioutput.onnx", "wb") as f:
#     f.write(onnx_model.SerializeToString())

# print("✅ Model exported as optuna_rf_multioutput.onnx")


In [49]:
# Print the order of box dimension columns used as targets
print("📦 X_train columns (model output order):")
for col in X_train.columns:
    print(col)


📦 X_train columns (model output order):
Total No. Of Quantity
Avg Quantity
Max Quantity
Category_100cc
Category_1100ml
Category_12 weight|kg
Category_150g
Category_4240cc
Category_500G
Category_5lbs
Category_710ml
Category_8150ml
Category_900ml
Category_C box 16*12*11.5


In [50]:
# Print the order of box dimension columns used as targets
print("📦 y_train columns (model output order):")
for col in y_train.columns:
    print(col)


📦 y_train columns (model output order):
10x10x10
12x18x10
12x8x20
18x17x27
20x20x34
30x30x29
32x20x17
38x30x20
40x30x29
40x30x34
40x40x34
42x42x30
48x34x27
60x30x30
8x22x13
8x8x13
